In [1]:
%reload_ext autoreload
%autoreload 2
%config InlineBackend.figure_format = 'retina'

In [2]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [3]:
import keras
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from IPython.display import clear_output
from plot_losses import PlotLosses

Using TensorFlow backend.


## Keras modules

In [4]:
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint 
from keras.models import Sequential
from keras.layers.core import Dense, Flatten
from keras.layers import Conv2D, AveragePooling2D

## Cargo datos

In [5]:
folder = '../input/fashion-mnist-itba-lab-ml-2018b/'
!ls {folder}

ls: ../input/fashion-mnist-itba-lab-ml-2018b/: No such file or directory


In [6]:
x = np.load(folder + 'train_images.npy')
y = np.loadtxt(folder + 'train_labels.csv', delimiter=',', skiprows=1)
x_test = np.load(folder + 'test_images.npy')

FileNotFoundError: [Errno 2] No such file or directory: '../input/fashion-mnist-itba-lab-ml-2018b/train_images.npy'

## Lo llevo a formato de imágenes de 28 x 28 x 1

In [ ]:
x = np.expand_dims(x, axis=3)
x.shape

In [ ]:
x_test = np.expand_dims(x_test, axis=3)
x_test.shape

In [ ]:
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size = 0.1)

## Paso a float

In [ ]:
x_train = x_train.astype('float32')
x_valid = x_valid.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_valid /= 255
x_test /= 255

In [ ]:
y_train_categorical = to_categorical(y_train)
y_val_categorical = to_categorical(y_valid)

## Red neuronal: CNN LeNet-5

In [ ]:
model = Sequential()

model.add(Conv2D(filters=6, kernel_size=(5, 5), activation='tanh', input_shape=(28,28,1), padding='same'))

model.add(AveragePooling2D())

model.add(Conv2D(filters=16, kernel_size=(5, 5), activation='tanh'))

model.add(AveragePooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(units=120, activation='tanh'))

model.add(Dense(units=84, activation='tanh'))

model.add(Dense(units=10, activation = 'softmax'))

model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

model.summary()

In [ ]:
model.compile(loss = 'categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Callbacks
## Callback para graficar
plot_losses = PlotLosses(plot_interval=1, evaluate_interval=20, x_val=x_valid, y_val_categorical=y_val_categorical)
## Callback para guardar pesos
checkpointer = ModelCheckpoint(filepath='checkpoint.mnist-cnn.hdf5', verbose=1, save_best_only=True)

In [ ]:
x_train.shape

In [ ]:
y_train_categorical.shape

In [ ]:
batch_size = 128
epocs = 10

model.fit(x_train, 
        y_train_categorical,
        epochs=epocs, batch_size=batch_size, 
        verbose=1, 
        validation_data = (x_valid, y_val_categorical),
        callbacks=[plot_losses, checkpointer],
    )

In [ ]:
model.load_weights('checkpoint.mnist-cnn.hdf5')

In [ ]:
loss, acc = model.evaluate(x_valid, y_val_categorical)

In [ ]:
print(acc)

In [ ]:
test_prediction = model.predict(x_test)

In [ ]:
test_labels = np.argmax(test_prediction, axis = 1)

In [ ]:
import pandas as pd
df = pd.DataFrame(data={"Category": test_labels}).astype(int)
df.to_csv("./submission.csv", sep=',',index=True,  index_label='Id')